In [59]:
import pandas as pd

ratings = pd.read_csv('./sample_data/Book-Ratings.csv', encoding='cp1252', sep=';')
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [61]:
books = pd.read_csv('./sample_data/Books.csv', encoding='cp1252', sep=';')
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [62]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy

from surprise.model_selection import train_test_split

In [63]:

# 데이터만 포함된 새로운 CSV 파일로 저장합니다.
ratings.to_csv('./sample_data/Book-Ratings-noheader.csv', index=False, header=False)


In [79]:
from surprise import Reader

reader = Reader(line_format='user item rating', sep=',', rating_scale=(1, 10))


print(reader)

In [86]:
data = Dataset.load_from_file('./sample_data/Book-Ratings-noheader.csv', reader=reader)

In [ ]:
train_set, test_set = train_test_split(data, test_size=0.25, random_state=0)

svd = SVD(n_factors=100, random_state=0)

svd.fit(train_set)
predictions = svd.test(test_set)
accuracy.rmse(predictions)

In [ ]:
from surprise.model_selection import cross_validate

In [87]:
from surprise.dataset import DatasetAutoFolds

data_folds = DatasetAutoFolds('./sample_data/Book-Ratings-noheader.csv', reader=reader)

In [88]:
trainset = data_folds.build_full_trainset()
print(trainset)

In [ ]:
svd = SVD(n_epochs=100, n_factors=50, random_state=0)
svd.fit(trainset)

In [ ]:
def get_unseen_surprise(ratings, books, userId):
    seen_books = ratings[ratings['User-ID'] == userId]['ISBN'].tolist()
    
    total_books = books['ISBN'].tolist()
    
    unseen_books = [book for book in total_books if book not in seen_books]
    print('평점 매긴 도서 수 : ', len(seen_books), ', 추천 대상 도서 수 : ', len(unseen_books),
          ', 전체 도서 수 : ', len(total_books))
    return unseen_books
nuseen_books = get_unseen_surprise(ratings, books, 276754)

In [94]:
def recomm_movie_by_surprise(svd, userId, unseen_books, top_n=10):
    predictions = [svd.predict(str(userId), str(ISBN)) for movieId in unseen_books]
    
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est() 변환값의 내림차순으로 정렬 수행
    predictions.sort(key=sortkey_est, reverse=True)
    
    top_predictions = predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출 : iid => movieId
    top_book_ids = [int(pred.iid) for pred in top_predictions]
    top_book_rating = [pred.est for pred in top_predictions] # 예측 평점 리스트 추출
    top_book_titles = books[books.ISBN.isin(top_book_ids)]['Book-Title']
    
    top_book_preds = [ (id, title, rating) for id, title, rating in zip(top_book_ids, top_book_titles, top_book_rating)]
    
    return top_book_preds


In [ ]:
# 사용자Id가 276754인 사용자의 10개 책 추천
unseen_books = get_unseen_surprise(ratings, books, 276754)
top_book_preds = recomm_movie_by_surprise(svd, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 책 리스트 ####')
for top_book in top_book_preds:
    print(top_book[1], ":" , top_book[2])